# Wstęp do Sztucznej Inteligencji - rok akademicki 2018/2019

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `{NrAlbumu}_{Nazwisko}_{Imie}_{PoprzedniaNazwa}`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia, w szczególności, że uzupełniłeś wszystkie pola `YOUR CODE HERE` oraz `YOUR ANSWER HERE`.

## Zadanie 1  (obowiązkowe, 5pkt.)
Zapożyczone z ćwiczeń do wykładu [Arabas, Cichosz](http://wazniak.mimuw.edu.pl/index.php?title=Sztuczna_inteligencja/SI_%C4%86wiczenia_4)

### Zamodeluj i odpowiedz na pytania.

W śledztwie dotyczącym zabójstwa inspektor Bayes rozważa dwie hipotezy:

- $H_1$ główny podejrzany zabił,
- $H_2$ główny podejrzany nie zabił, 

oraz następujące możliwe fakty:

- $E_1$ na miejscu zbrodni znaleziono odciski palców głównego podejrzanego,
- $E_2$ główny podejrzany nie ma alibi na czas popełnienia zabójstwa,
- $E_3$ główny podejrzany miał motyw zabicia ofiary,
- $E_4$ główny podejrzany był widziany w sądziedztwie miejsca, w którym mieszka nielegalny handlarz bronią,
- $E_5$ świadek zbrodni podał rysopis zabójcy nie pasujący do głównego podejrzanego. 

Zależności między takimi faktami a hipotezami opisują następujące prawdopodobieństwa:

$P(E_1|H_1)=0.7,\qquad P(E_1|H_2)=0.3,$

$P(E_2|H_1)=0.8,\qquad P(E_2|H_2)=0.4,$

$P(E_3|H_1)=0.9,\qquad P(E_3|H_2)=0.5,$

$P(E_4|H_1)=0.4,\qquad P(E_4|H_2)=0.2,$

$P(E_5|H_1)=0.2,\qquad P(E_5|H_2)=0.4.$ 

__W którym przypadku prawdopodobieństwo popełnienia zabójstwa byłoby największe?__

1. Gdyby znaleziono na miejscu zbrodni jego odciski palców.
2. Gdyby stwierdzono, że nie miał alibi i miał motyw.
3. Gdyby znaleziono na miejscu zbrodni jego odciski palców oraz stwierdzono, że był widziany w sąsiedztwie miejsca, w którym mieszka nielegalny handlarz bronią, ale świadek zbrodni podał rysopis zabójcy nie pasujący do głównego podejrzanego.

#### TWÓJ PROGRAM:

In [14]:
import pymc3 as pm
import numpy as np

with pm.Model() as zab_model:
    zabil = pm.Bernoulli('zabil', 0.5)
    
    p_odciski = pm.Deterministic('p_odciski', pm.math.switch(zabil, 0.7, 0.3))
    odciski = pm.Bernoulli('odciski', p_odciski)
    
    p_niealibi = pm.Deterministic('p_niealibi', pm.math.switch(zabil, 0.8, 0.4))
    niealibi = pm.Bernoulli('niealibi', p_niealibi)
    
    p_motyw = pm.Deterministic('p_motyw', pm.math.switch(zabil, 0.9, 0.5))
    motyw = pm.Bernoulli('motyw', p_motyw)
    
    p_widziany = pm.Deterministic('p_widziany', pm.math.switch(zabil, 0.4, 0.2))
    widziany = pm.Bernoulli('widziany', p_widziany)
    
    p_nierysopis = pm.Deterministic('p_nierysopis', pm.math.switch(zabil, 0.2, 0.4))
    nierysopis = pm.Bernoulli('nierysopis', p_nierysopis)
with zab_model:
    trace = pm.sample(50000, chains=1)
# print(trace)
# print(trace.varnames)
# trace.get_values('zabil')
# axs = pm.traceplot(trace, varnames=['zabil', 'odciski', 'niealibi', 'motyw', 'widziany', 'nierysopis'])
p_h1_e1 = (trace['zabil']*trace['odciski']).sum()/trace['odciski'].sum()

p_h1_e1ANDe2 = (trace['zabil']*trace['motyw']*trace['niealibi']).sum()/((trace['zabil']*trace['motyw']*trace['niealibi']).sum() + (np.logical_not(trace['zabil'])*trace['motyw']*trace['niealibi']).sum())

p_h1_e1ANDe4ANDe5 = (trace['zabil']*trace['odciski']*trace['widziany']*trace['nierysopis']).sum()/((trace['zabil']*trace['odciski']*trace['widziany']*trace['nierysopis']).sum() + (np.logical_not(trace['zabil'])*trace['odciski']*trace['widziany']*trace['nierysopis']).sum())

print(p_h1_e1, p_h1_e1ANDe2, p_h1_e1ANDe4ANDe5, sep='\n')

p_zabil_odciski = (trace['zabil']*trace['odciski']).sum()/trace['odciski'].sum()
p_zabil_alibi_motyw = (trace['zabil']*trace['niealibi']*trace['motyw']).sum()/(trace['niealibi']*trace['motyw']).sum()
p_zabil_odciski_bron_rysopis = (trace['zabil']*trace['odciski']*trace['widziany']*trace['nierysopis']).sum()/(trace['odciski']*trace['widziany']*trace['nierysopis']).sum()
print('',p_zabil_odciski, p_zabil_alibi_motyw, p_zabil_odciski_bron_rysopis, sep='\n')

Sequential sampling (1 chains in 1 job)
BinaryGibbsMetropolis: [zabil, odciski, niealibi, motyw, widziany, nierysopis]
100%|██████████████████████████████████████████████████████████████████████████| 50500/50500 [00:28<00:00, 1781.66it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.6949930181528027
0.7797247105090671
0.6992628992628993

0.6949930181528027
0.7797247105090671
0.6992628992628993


#### ODPOWIEDŹ:

zad1 0.7066194605776116
zad2 0.826449149600833
zad3 0.7012012012012012

## Zadanie 2  (obowiązkowe, 5pkt.)

### Zamodeluj i odpowiedz na pytania.
System alarmowy w mieszkaniu, reaguje na włamania oraz, niestety, również na drobne trzęsienia (ziemi). Sąsiedzi John i Mary są umówieni, żeby zadzwonić do właściciela gdy usłyszą alarm. John jest nadgorliwy i bierze różne zdarzenia (np. dzwonek telefonu) za sygnał alarmowy (i wtedy zawsze dzwoni). Mary rozpoznaje alarm poprawnie, lecz często słucha głośnej muzyki i może go w ogóle nie usłyszeć. 

Sieć przekonań dla systemu alarmowego wygląda następująco:
![bsiec.PNG](http://torus.uck.pk.edu.pl/~amarsz/images/bsiec.PNG)

__Jakie jest prawdopodobieństwo, że:__
1. włączy się alarm?
2. doszło do włamanie jeśli wiadom, że włączył się alarm?
3. zdarzyło się trzęsienie ziemi jeśli wiadomo, żę włączył się alarm?
1. w razie włamania ktoś zadzwoni?
2. zawiadomienie o włamaniu jest fałszywe?
3. rozległ się alarm, przy czym nie wystąpiło ani trzęsienie ziemi ani włamanie, ale oboje John i Mary zadzwonili? (prawd. bezwarunkowe)

#### TWÓJ PROGRAM:

In [12]:
import pymc3 as pm
import numpy as np

with pm.Model() as model:
    buglary = pm.Bernoulli('buglary', 0.001)
    earthquake = pm.Bernoulli('earthquake', 0.002)
    p_alarm = pm.Deterministic('p_alarm', pm.math.switch(buglary, pm.math.switch(earthquake, 0.95, 0.94), pm.math.switch(earthquake, 0.29, 0.001)))
    alarm = pm.Bernoulli('alarm', p_alarm)
    p_john = pm.Deterministic('p_john', pm.math.switch(alarm, 0.9, 0.05))
    john = pm.Bernoulli('john', p_john)
    p_mary = pm.Deterministic('p_mary', pm.math.switch(alarm, 0.7, 0.01))
    mary = pm.Bernoulli('mary', p_mary)
    
    trace = pm.sample(200000, chains=1)

p_a = (trace['alarm']).sum()/len(trace['alarm'])

p_b_a = (trace['buglary']*trace['alarm']).sum()/(trace['alarm']).sum()

p_e_a = (trace['earthquake']*trace['alarm']).sum()/(trace['alarm']).sum()

p_wlamanie = (trace['buglary'] * trace['john']).sum()/(trace['buglary']).sum() + (trace['buglary'] * trace['mary']).sum()/(trace['buglary']).sum() - (trace['buglary'] * trace['mary'] * trace['john']).sum()/(trace['buglary']).sum() 

zad5 = (trace['alarm']*np.logical_not(trace['buglary'])).sum()/np.logical_not(trace['buglary']).sum()

zad6 = (trace['alarm']*np.logical_not(trace['buglary'])*np.logical_not(trace['earthquake'])).sum() /(np.logical_not(trace['buglary'])*np.logical_not(trace['earthquake'])).sum() * trace['john'].sum()/len(trace['john']) * trace['mary'].sum()/len(trace['mary'])
print(p_a, p_b_a, p_e_a, p_wlamanie, zad5, zad6, sep = '\n')

Sequential sampling (1 chains in 1 job)
BinaryGibbsMetropolis: [buglary, earthquake, alarm, john, mary]
100%|████████████████████████████████████████████████████████████████████████| 200500/200500 [01:26<00:00, 2309.70it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.002485


0.3682092555331992
0.2012072434607646
0.9128205128205129
0.0015715322439378394
6.412972048381273e-07


#### ODPOWIEDŹ:

zad1 0.002485
zad2 0.3682092555331992
zad3 0.2012072434607646
zad4 0.9128205128205129
zad5 0.0015715322439378394
zad6 6.412972048381273e-07

<div style="text-align: right">&copy; Zakład Inteligencji Obliczeniowej, Instytut Informatyki, Politechnika Krakowska </div>